In [1]:
import sys
sys.path.append("../")
from utils import *

In [2]:
with open(data_path + "train_test_data.pkl", "rb") as f:
    X_train, y_train, X_test, y_test = pickle.load(f)

In [3]:
d = {'pl':pd.read_csv(data_path+'pl_tracks.csv'),\
     't':pd.read_csv(data_path+'tracks.csv')}

In [4]:
# Calculate popularity: number of occurances
pop = d['pl'].groupby('track_uri').count().reset_index()
pop.columns = ['track_uri', 'popularity']
df = pd.merge(d['t'], pop, on = 'track_uri', how = 'left')
df.head()

track_name  duration_ms  \
0  Lose Control (feat. Ciara & Fat Man Scoop)       226863   
1                                       Toxic       198800   
2                               Crazy In Love       235933   
3                              Rock Your Body       267266   
4                                It Wasn't Me       227600   

                              track_uri  \
0  spotify:track:0UaMYEvWZi0ZqiDOoHU3YI   
1  spotify:track:6I9VzXrHxO9rA9A5euc8Ak   
2  spotify:track:0WqIKmW4BTrj3eJFmnCKMv   
3  spotify:track:1AWQoqb9bSvzTjaLralEkT   
4  spotify:track:1lzr43nnXAijIGYnCT8M8H   

                              artist_uri  \
0  spotify:artist:2wIVse2owClT7go1WT98tk   
1  spotify:artist:26dSoYclwsYLMAKD3tpOr4   
2  spotify:artist:6vWDO969PvNqNYHIOW5v0m   
3  spotify:artist:31TPClRtHm23RisEBtV3X7   
4  spotify:artist:5EvFsr3kj42KNv97ZEnqij   

                              album_uri  popularity  
0  spotify:album:6vV5UrXcfyQD1wu4Qo2I9K        6841  
1  spotify:album:0z7pVBGOD7HCIB7S8eLkLI       13205  
2  spotify:album:25hVFAxTlDvXbx2X2QkUkE       16914  
3  spotify:album:6QPkyl04rXwTGlGlcYaRoW        9066  
4  spotify:album:6NmFmPX56pcLBOFMhIiKvF       23173

In [5]:
# Define the list of track_uri ordered by popularity
pop_sorted_tracks = df.sort_values('popularity', ascending = False)['track_uri'].values.tolist()

In [6]:
def popularity_recommender(data_train, test_size = 5):
    ''' A naive recommender system that outputs the most popular <test_size> songs
        that is not currently in the album. '''
    preds = []
    for pid,visited in data_train.items():
        cur_preds = []
        for track in pop_sorted_tracks:
            if track not in visited:
                cur_preds.append(track)
            if len(cur_preds) == test_size:
                preds.append(cur_preds)
                break
    return preds

In [7]:
def overlap_score(tracks, pred_tracks, test_size = 5):
    ''' Computes the overlap score for tracks and pred_tracks. 
        returns #overlap'''
    assert len(tracks) == len(pred_tracks)
    return sum([a in tracks for a in pred_tracks])

def avg_overlap(true_dict, pred):
    ''' Returns the accuracy score given true_label and pred'''
    assert len(true_dict) == len(pred)
    avg_overlap = np.mean([overlap_score(a, b) for a,b in zip(true_dict.values(), preds)])    
    return avg_overlap

In [9]:
preds = popularity_recommender(X_train)
print("Average overlap score is:", avg_overlap(y_train, preds))

Average overlap score is: 0.03294367693942614


In [10]:
preds = popularity_recommender(X_test)
print("Average overlap score is:", avg_overlap(y_test, preds))

Average overlap score is: 0.03319838056680162
